# Load Data

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset


In [ ]:
# Datasets path
data_path = 'data' # NOTE: Change this path to the location of your BATADAL dataset files

# Load benign train data
train_benign_data = pd.read_csv(os.path.join(data_path, 'BATADAL_dataset03.csv')) # clean data

# Load attack train data
train_attack_data = pd.read_csv(os.path.join(data_path, 'BATADAL_dataset04.csv')) # data with attacks: 6-months long; attacks 1-7
train_attack_data = train_attack_data.rename(
                                columns={' L_T1': 'L_T1', ' L_T2': 'L_T2', ' L_T3': 'L_T3', ' L_T4': 'L_T4', ' L_T5': 'L_T5', ' L_T6': 'L_T6',	' L_T7': 'L_T7',
                                            ' P_J14': 'P_J14', ' P_J422': 'P_J422', ' P_J280': 'P_J280', ' P_J269': 'P_J269', ' P_J300': 'P_J300', ' P_J256': 'P_J256', ' P_J289': 'P_J289', ' P_J415': 'P_J415', ' P_J302': 'P_J302', ' P_J306': 'P_J306', ' P_J307': 'P_J307', ' P_J317': 'P_J317',
                                            ' F_PU1': 'F_PU1', ' F_PU2': 'F_PU2', ' F_PU3': 'F_PU3', ' F_PU4': 'F_PU4', ' F_PU5': 'F_PU5', ' F_PU6': 'F_PU6', ' F_PU7': 'F_PU7', ' F_PU8': 'F_PU8', ' F_PU9': 'F_PU9', ' F_PU10': 'F_PU10', ' F_PU11': 'F_PU11', ' F_V2': 'F_V2',
                                            ' S_PU1': 'S_PU1', ' S_PU2': 'S_PU2', ' S_PU3': 'S_PU3', ' S_PU4': 'S_PU4', ' S_PU5': 'S_PU5', ' S_PU6': 'S_PU6', ' S_PU7': 'S_PU7', ' S_PU8': 'S_PU8', ' S_PU9': 'S_PU9', ' S_PU10': 'S_PU10', ' S_PU11': 'S_PU11', ' S_V2': 'S_V2'})

# Load test data
test_data = pd.read_csv(os.path.join(data_path, 'BATADAL_test_dataset.csv')) # data with attacks: 3-months long; attacks 8-14

In [3]:
# Separate the DATETIME column and the features
train_benign_datetime = train_benign_data['DATETIME']
train_attack_datetime = train_attack_data['DATETIME']
test_datetime = test_data['DATETIME']

# Preprocessing: Scale data using MinMaxScaler
scaler = MinMaxScaler()
train_benign_scaled = scaler.fit_transform(train_benign_data.drop(columns=['DATETIME', 'ATT_FLAG']))
train_attack_scaled = scaler.transform(train_attack_data.drop(columns=['DATETIME', ' ATT_FLAG']))
test_scaled = scaler.transform(test_data.drop(columns=['DATETIME']))

# Convert data to PyTorch tensors
train_benign_tensor = torch.tensor(train_benign_scaled, dtype=torch.float32)
train_attack_tensor = torch.tensor(train_attack_scaled, dtype=torch.float32)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)


In [4]:
# Attack intervals
train_benign_attacks_time_intervals = []
train_attack_attacks_time_intervals = [
        {"id": 1, "start": "13/09/2016 23", "end": "16/09/2016 00"},    # Attacker changesL_T7 thresholds -->> low level in T7
        {"id": 2, "start": "26/09/2016 11", "end": "27/09/2016 10"},    # Attacker changesL_T7 thresholds -->> T2 low level in T7
        {"id": 3, "start": "09/10/2016 09", "end": "11/10/2016 20"},   # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 4, "start": "29/10/2016 19", "end": "02/11/2016 16"},   # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 5, "start": "26/11/2016 17", "end": "29/11/2016 04"},   # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 6, "start": "06/12/2016 07", "end": "10/12/2016 04"},   # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 7, "start": "14/12/2016 15", "end": "19/12/2016 04"},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
]
test_attacks_time_intervals = [
        {"id": 8, "start": "16/01/2017 00", "end": "19/01/2017 06"},    # Attacker changes L_T3 thresholds -->> low level in T3
        {"id": 9, "start": "30/01/2017 08", "end": "02/02/2017 00"},    # Attacker changes L_T2 readings -->> T2 overflow
        {"id": 10, "start": "09/02/2017 03", "end": "10/02/2017 09"},   # Malicious activation of pump PU3
        {"id": 11, "start": "12/02/2017 01", "end": "13/02/2017 07"},   # Malicious activation of pump PU3
        {"id": 12, "start": "24/02/2017 05", "end": "28/02/2017 08"},   # Attacker changes L_T2, F_V2 and S_V2, P_J14 and P_J422 -->> T2 overflow
        {"id": 13, "start": "10/03/2017 14", "end": "13/03/2017 21"},   # Attacker changes L_T7 thresholds -->> PU10/PU11 are switching on/off continuously
        {"id": 14, "start": "25/03/2017 20", "end": "27/03/2017 01"}    # Attacker changes L_T4 readings -->> T6 overflow
]

# Convert attack intervals to datetime objects
train_benign_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in train_benign_attacks_time_intervals
]
train_attack_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in train_attack_attacks_time_intervals
]
test_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in test_attacks_time_intervals
]

# Attack indexes
train_benign_attacks_index_intervals = []
train_attack_attacks_index_intervals = [
        {"id": 1, "start": 1727, "end": 1776},    # Attacker changesL_T7 thresholds -->> low level in T7
        {"id": 2, "start": 2027, "end": 2050},    # Attacker changesL_T7 thresholds -->> T2 low level in T7
        {"id": 3, "start": 2337, "end": 2396},    # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 4, "start": 2827, "end": 2920},    # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 5, "start": 3497, "end": 3556},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 6, "start": 3727, "end": 3820},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 7, "start": 3927, "end": 4036},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
]
test_attacks_index_intervals = [
        {"id": 8, "start": 297, "end": 366},    # Attacker changes L_T3 thresholds -->> low level in T3
        {"id": 9, "start": 632, "end": 696},    # Attacker changes L_T2 readings -->> T2 overflow
        {"id": 10, "start": 867, "end": 897},   # Malicious activation of pump PU3
        {"id": 11, "start": 937, "end": 967},   # Malicious activation of pump PU3
        {"id": 12, "start": 1229, "end": 1328},   # Attacker changes L_T2, F_V2 and S_V2, P_J14 and P_J422 -->> T2 overflow
        {"id": 13, "start": 1574, "end": 1653},   # Attacker changes L_T7 thresholds -->> PU10/PU11 are switching on/off continuously
        {"id": 14, "start": 1940, "end": 1969}    # Attacker changes L_T4 readings -->> T6 overflow
]

train_benign_attacks_indices  = np.zeros(train_benign_tensor.shape[0])
for attack in train_benign_attacks_index_intervals:
    train_benign_attacks_indices[attack['start']:attack['end']+1] = 1
train_attack_attacks_indices  = np.zeros(train_attack_tensor.shape[0])
for attack in train_attack_attacks_index_intervals:
    train_attack_attacks_indices[attack['start']:attack['end']+1] = 1
test_attacks_indices  = np.zeros(test_tensor.shape[0])
for attack in test_attacks_index_intervals:
    test_attacks_indices[attack['start']:attack['end']+1] = 1


In [5]:
# Add column to indicate if the network is attacked or not

# Training data - benign
new_column = torch.zeros(train_benign_tensor.shape[0], 1)  # Create a column of zeros and ones with the same number of rows
for attack in train_benign_attacks_index_intervals:
    new_column[attack["start"]:attack["end"]+1] = 0
train_benign_tensor = torch.cat((train_benign_tensor, new_column), dim=1) # Concatenate along the column dimension (dim=1)

# Training data - attack
new_column = torch.zeros(train_attack_tensor.shape[0], 1)  # Create a column of zeros and ones with the same number of rows
for attack in train_attack_attacks_index_intervals:
    new_column[attack["start"]:attack["end"]+1] = 1
train_attack_tensor = torch.cat((train_attack_tensor, new_column), dim=1) # Concatenate along the column dimension (dim=1)

# Testing data
new_column = torch.zeros(test_tensor.shape[0], 1)  # Create a column of zeros and ones with the same number of rows
for attack in test_attacks_index_intervals:
    new_column[attack["start"]:attack["end"]+1] = 1
test_tensor = torch.cat((test_tensor, new_column), dim=1) # Concatenate along the column dimension (dim=1)


In [6]:
# Dataset Class for Time Series Windowing
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        return (self.data[idx:idx + self.window_size], self.data[idx + self.window_size])


# Parameters
window_sizes = [1,2,8,16,32]
batch_size = 64

# Create datasets and data loaders
train_benign_dataset = []
train_attack_dataset = []
test_dataset = []
train_benign_loader = []
train_attack_loader = []
test_loader = []
for idx, window_size in enumerate(window_sizes):
  # Create datasets
  train_benign_dataset.append(TimeSeriesDataset(train_benign_tensor, window_size))
  train_attack_dataset.append(TimeSeriesDataset(train_attack_tensor, window_size))
  test_dataset.append(TimeSeriesDataset(test_tensor, window_size))

  # Create data loaders
  train_benign_loader.append(DataLoader(train_benign_dataset[idx], batch_size=batch_size, shuffle=True))
  train_attack_loader.append(DataLoader(train_attack_dataset[idx], batch_size=batch_size, shuffle=True))
  test_loader.append(DataLoader(test_dataset[idx], batch_size=1, shuffle=False))


In [ ]:
input_dim = train_benign_scaled.shape[1]

print("Number of features: ", input_dim)
print(f"Benign train data shape: {train_benign_tensor.shape}")
print(f"Attack aware train data shape: {train_attack_tensor.shape}")
print(f"Test data shape: {test_tensor.shape}")

# AutoEncoders

In [23]:
import torch.nn as nn

###################### Transformer-based Autoencoder #######################
# Define Transformer-based Autoencoder
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, ff_dim, num_layers):
        super(TransformerAutoencoder, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, batch_first=True)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)

        self.embedding = nn.Linear(input_dim, embed_dim)
        self.output_layer = nn.Linear(embed_dim, input_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        encoded = self.encoder(embedded)
        decoded = self.decoder(encoded, embedded)
        output = self.output_layer(decoded)
        return output

###################### Convolutional Recurrent Autoencoder #######################
# Define Convolutional Recurrent Autoencoder
class ConvRecurrentAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers):
        super(ConvRecurrentAutoencoder, self).__init__()

        # Encoder: Convolutional Layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=kernel_size, padding=kernel_size // 2)
        self.relu = nn.ReLU()

        # Encoder: GRU Layers
        self.gru_encoder = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)

        # Decoder: GRU Layers
        self.gru_decoder = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)

        # Decoder: Deconvolutional Layers
        self.deconv1 = nn.ConvTranspose1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=kernel_size, padding=kernel_size // 2)
        self.deconv2 = nn.ConvTranspose1d(in_channels=hidden_dim, out_channels=input_dim, kernel_size=kernel_size, padding=kernel_size // 2)

    def forward(self, x):
        # Encoder: Convolutional Layers
        # Input shape: (batch_size, sequence_length, input_dim)
        x = x.permute(0, 2, 1)  # Switch to (batch_size, input_dim, sequence_length) for Conv1d
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))

        # Encoder: GRU Layers
        # Input shape after conv: (batch_size, hidden_dim, sequence_length)
        x = x.permute(0, 2, 1)  # Switch to (batch_size, sequence_length, hidden_dim) for GRU
        _, h = self.gru_encoder(x)

        # Decoder: GRU Layers
        decoded, _ = self.gru_decoder(x, h)

        # Decoder: Deconvolutional Layers
        # Output shape after GRU: (batch_size, sequence_length, hidden_dim)
        decoded = decoded.permute(0, 2, 1)  # Switch to (batch_size, hidden_dim, sequence_length) for ConvTranspose1d
        decoded = self.relu(self.deconv1(decoded))
        output = self.deconv2(decoded)

        return output.permute(0, 2, 1)  # Return to (batch_size, sequence_length, input_dim)

####################### Fully Connected Autoencoder #######################
# Define Fully Connected Autoencoder based on the paper
class FullyConnectedAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers):
        super(FullyConnectedAutoencoder, self).__init__()

        # Encoder: Fully Connected Layers with Tanh activations
        encoder_layers = []
        current_dim = input_dim
        for _ in range(num_layers - 1):
            encoder_layers.append(nn.Linear(current_dim, hidden_dim))
            encoder_layers.append(nn.Tanh())
            current_dim = hidden_dim
        # Final layer to latent space
        encoder_layers.append(nn.Linear(current_dim, latent_dim))
        encoder_layers.append(nn.Tanh())
        self.encoder = nn.Sequential(*encoder_layers)

        # Decoder: Fully Connected Layers with Tanh activations
        decoder_layers = []
        current_dim = latent_dim
        for _ in range(num_layers - 1):
            decoder_layers.append(nn.Linear(current_dim, hidden_dim))
            decoder_layers.append(nn.Tanh())
            current_dim = hidden_dim
        # Final layer to reconstruct the input
        decoder_layers.append(nn.Linear(current_dim, input_dim))
        decoder_layers.append(nn.Tanh())  # Assuming input data is scaled between -1 and 1
        self.decoder = nn.Sequential(*decoder_layers)

    def forward(self, x):
        # Flatten the input if necessary (assuming input shape is (batch_size, sequence_length, input_dim))
        batch_size, sequence_length, input_dim = x.shape
        x = x.view(batch_size * sequence_length, input_dim)  # Flatten input to (batch_size * sequence_length, input_dim)

        # Encoder
        encoded = self.encoder(x)

        # Decoder
        decoded = self.decoder(encoded)

        # Reshape the output back to the original format (batch_size, sequence_length, input_dim)
        output = decoded.view(batch_size, sequence_length, input_dim)

        return output


# Weight the Loss Function
Use a weighted loss function to penalize reconstruction errors differently for normal and anomalous data:

*   Assign a higher weight to reconstruction errors for normal data, encouraging the model to focus on learning the normal pattern.
*   Assign a lower weight to reconstruction errors for anomalous data, preventing the autoencoder from learning to reconstruct anomalies effectively.

In [9]:
# Compute loss
from torch import nn

def custom_loss(outputs, inputs, normal_weight=1, anomaly_weight=1):
    if anomaly_weight==None:
        loss_f = nn.MSELoss()
        return loss_f(outputs, inputs[:,:, :-1])
    else:
        is_anomaly = inputs[:, :, -1]  # the last dimension indicates if it's an anomaly

        is_not_anomaly_view = (1 - is_anomaly).view(-1)
        is_not_anomaly_view = is_not_anomaly_view[:,None]

        is_anomaly_view = is_anomaly.view(-1)
        is_anomaly_view = is_anomaly_view[:,None]

        mse_view = ((inputs[:,:, :-1] - outputs)**2).view(-1, 43)

        loss_normal = normal_weight * (is_not_anomaly_view * mse_view).mean()
        loss_anomaly = anomaly_weight * (is_anomaly_view * mse_view).mean()

        return loss_normal + loss_anomaly


# Smooth the reconstruction error

Postprocessing: an additional step - smoothing of anomaly scores (sliding averages) before the attack detection. The smoothing is performed by forming  of the anomaly score values preceding in time.

In [10]:
filter_sizes = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29]

def smooth_models_reconstruction_errors(reconstruction_errors):
  reconstruction_errors_for_filter_size = {}

  for filter_size in filter_sizes:
    reconstruction_errors_for_filter_size[filter_size] = []

    l = len(reconstruction_errors)
    for idx in range(l):
      sublst = reconstruction_errors[max((idx-filter_size+1),0): min((idx+1), l)]
      avg_val = sum(sublst) / len(sublst)

      reconstruction_errors_for_filter_size[filter_size].append(avg_val)

  return reconstruction_errors_for_filter_size


# Training and Testing Loops

In [11]:
from torch import nn
import json
from torch.optim import Adam
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, auc

In [12]:
# Training loop
# NOTE: for anomaly_weight=None we have benign model
def train_model(model, train_loader, criterion, optimizer, epochs, anomaly_weight, device, print_progress=True):
    model.train()

    total_loss = None
    for epoch in range(epochs):
        total_loss = 0

        for batch in train_loader:
            inputs, _ = batch
            inputs = inputs.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs[:,:,:-1])
            loss = criterion(outputs, inputs, anomaly_weight=anomaly_weight)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if print_progress:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.6f}')

    return model, total_loss

In [13]:
# Evaluating loop
def evaluate_model(model, test_loader, device):
    model.eval()
    loss_f = nn.MSELoss()
    
    reconstruction_errors = []
    with torch.no_grad():
        for batch in test_loader:
            inputs, _ = batch
            inputs = inputs.float().to(device)  # Ensure inputs are of the correct type (float32)
            outputs = model(inputs[:,:,:-1])   # Get model output
            loss = loss_f(outputs, inputs[:,:, :-1]).item()  # Calculate MSE for the batch
            reconstruction_errors.append(loss)

    return reconstruction_errors

In [14]:
# Training and evaluation loop
# for anomaly_weight=None we have benign model
def train_and_evaluate(model, train_loader, test_loader, optimizer, epochs, anomaly_weight, device):
    criterion = custom_loss

    # Training loop
    model, total_loss = train_model(model, train_loader, criterion, optimizer, epochs, anomaly_weight, device, print_progress=False)

    # Evaluation loop
    reconstruction_errors = evaluate_model(model, test_loader, device)

    # Return total loss and average validation loss
    return model, total_loss, sum(reconstruction_errors) / len(test_loader)


In [15]:
# Training and evaluation and save loop
# for anomaly_weight=None we have benign model
def train_and_evaluate_and_save(model, models_path, model_name, train_loader, test_loader, optimizer, epochs, anomaly_weight, device):
    criterion = custom_loss

    # Training 
    model, _ = train_model(model, train_loader, criterion, optimizer, epochs, anomaly_weight, device, print_progress=False)

    # Evaluation
    reconstruction_errors = evaluate_model(model, test_loader, device)
    smooth_reconstruction_errors = smooth_models_reconstruction_errors(reconstruction_errors)

    # Save
    torch.save(model, models_path + f'{model_name}.pth')
    print(f'{model_name}.pth model saved')
    with open(models_path + f"reconstruction_errors_{model_name}.json", "w") as file:
      json.dump(reconstruction_errors, file)
      print(f"reconstruction errors saved to reconstruction_errors_{model_name}.json")
    with open(models_path + f"smooth_reconstruction_errors_{model_name}.json", "w") as file:
      json.dump(smooth_reconstruction_errors, file)
      print(f"smooth reconstruction errors saved to smooth_reconstruction_errors_{model_name}.json")

    # Return
    return model, reconstruction_errors, smooth_reconstruction_errors

In [16]:

def train_all(models_path, models_type, best_params, anomaly_weight, train_loader, test_loader, window_size, input_dim, device):

  best_models = {}
  reconstruction_errors = {}
  smooth_reconstruction_errors = {}
  
  # Train models with best parameters
  best_models['TransformerAutoencoder'] = TransformerAutoencoder(
        input_dim=input_dim, 
        embed_dim=best_params['TransformerAutoencoder']['embed_dim'], 
        num_heads=best_params['TransformerAutoencoder']['num_heads'], 
        ff_dim=best_params['TransformerAutoencoder']['ff_dim'], 
        num_layers=best_params['TransformerAutoencoder']['num_layers']
  ).to(device)
  best_models['ConvRecurrentAutoencoder'] = ConvRecurrentAutoencoder(
      input_dim=input_dim,
      hidden_dim=best_params['ConvRecurrentAutoencoder']['hidden_dim'], 
      kernel_size=best_params['ConvRecurrentAutoencoder']['kernel_size'],
      num_layers=best_params['ConvRecurrentAutoencoder']['num_layers']
  ).to(device)
  best_models['FullyConnectedAutoencoder'] = FullyConnectedAutoencoder(
      input_dim=input_dim,
      hidden_dim=best_params['FullyConnectedAutoencoder']['hidden_dim'],
      latent_dim=best_params['FullyConnectedAutoencoder']['latent_dim'],
      num_layers=best_params['FullyConnectedAutoencoder']['num_layers']
    ).to(device)

  for ae_type in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
    print(f"\n#####   {ae_type}   #####")

    best_models[ae_type], reconstruction_errors[ae_type], smooth_reconstruction_errors[ae_type] = train_and_evaluate_and_save(
      model = best_models[ae_type], 
      models_path = models_path, 
      model_name = f"{models_type}_{ae_type}_ws{window_size}", 
      train_loader = train_loader, 
      test_loader = test_loader, 
      optimizer = Adam(best_models[ae_type].parameters(), lr=best_params[ae_type]['learning_rate']),
      epochs = best_params[ae_type]['epochs'], 
      anomaly_weight = anomaly_weight[ae_type],
      device = device)
    
    print(f"#####   END {ae_type}   #####")

  return best_models, reconstruction_errors, smooth_reconstruction_errors


In [17]:
def find_segments(arr):
  """
  Finds segments of consecutive True values in a boolean array.

  Args:
    arr: A boolean NumPy array.

  Returns:
    A list of tuples, where each tuple represents a segment and contains the
    start and end indices (inclusive) of the segment.
  """
  segments = []
  start = -1
  for i in range(len(arr)):
    if arr[i] and start == -1:
      start = i
    elif not arr[i] and start != -1:
      segments.append((start, i - 1, i - start))
      start = -1
  if start != -1:
    segments.append((start, len(arr) - 1, len(arr)))
  return segments


In [ ]:
def calculate_model_performance(test_ground_truth_labels, reconstruction_errors, method_name):

  # Calculate the best threshold to detect attacks using F1 score
  precision, recall, thresholds = precision_recall_curve(test_ground_truth_labels, reconstruction_errors)

  # Compute ROC curve
  fpr, tpr, _ = roc_curve(test_ground_truth_labels, reconstruction_errors)
  # Compute AUC (Area Under Curve)
  roc_auc = auc(fpr, tpr)

  # Calculate F1 score for each threshold
  f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
  best_threshold_idx = np.argmax(f1_scores)
  best_threshold = thresholds[best_threshold_idx]
  best_f1_score = f1_scores[best_threshold_idx]

  # Detect attacks using the best threshold
  detected_attacks = reconstruction_errors > best_threshold

  # TP
  TP = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 1))
  # TN
  TN = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 0))
  # FP
  FP = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 0))
  # FN
  FN = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 1))

  # TPR
  TPR = TP/(TP+FN)
  # TNR
  TNR = TN/(FP+TN)

  # Sclf
  Sclf = (TPR + TNR)/2
  # Sttd
  Sttd = None

  # Calculate the percentage of missed attack samples (False Negatives)
  total_attack_samples = np.sum(test_ground_truth_labels == 1)
  missed_attacks = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 1))
  missed_attack_percentage = (missed_attacks / total_attack_samples) * 100

  # Calculate the percentage of wrongly detected samples (False Positives)
  total_clean_samples = np.sum(test_ground_truth_labels == 0)
  wrongly_detected = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 0))
  wrongly_detected_percentage = (wrongly_detected / total_clean_samples) * 100


  # Add results to the table
  new_row = {
    "num_of_detected": 1,
    "TP": TP,
    "FP": FP,
    "TN": TN,
    "FN": FN,
    "TPR": TPR,
    "TNR": TNR,
    "S": None, #y*Sttd+(1-y)*Sclf
    "Sttd": Sttd,
    "Sclf": Sclf,
    "total_attacks": total_attack_samples,
    "total_clean": total_clean_samples,
    "missed_attacks": missed_attack_percentage,
    "wrongly_detected": wrongly_detected_percentage,
    "accuracy": (TP+TN)/(TP+TN+FP+FN),
    "f1_scores": f1_scores,
    "best_f1_score": best_f1_score,
    "best_threshold": best_threshold,
    "precisuons": precision,
    "recalls": recall,
    "fpr": fpr,
    "tpr": tpr,
    "roc_auc": roc_auc,
  }
  new_row_df = pd.DataFrame([new_row], index=[method_name])

  return new_row_df, detected_attacks, precision, recall


# Plot

In [ ]:
import matplotlib.pyplot as plt

def plot_reconstruction_errors(attack_indices, attack_index_intervals, window_size, reconstruction_errors, sufix="", plot_threshold=False, threshold=None):
    # Plotting both reconstruction errors on a single plot
    plt.figure(figsize=(12, 6))

    # Adjust indices to account for the window size - First-Point Strategy
    adjusted_indices = attack_indices[:-window_size]

    for (errors, label, color) in reconstruction_errors:
        plt.plot(adjusted_indices, errors, label=label, color=color)

    # Shading attack intervals
    for interval in attack_index_intervals:
        start_idx = interval["start"]
        end_idx = interval["end"]

        # Adjust the indices to account for the window size
        if start_idx >= window_size and end_idx >= window_size:
            adjusted_start_idx = start_idx - window_size
            adjusted_end_idx = end_idx - window_size
            plt.axvspan(adjusted_start_idx, adjusted_end_idx, color='red', alpha=0.3)
        elif start_idx < window_size and end_idx >= window_size:
            adjusted_start_idx = 0
            adjusted_end_idx = end_idx - window_size
            plt.axvspan(adjusted_start_idx, adjusted_end_idx, color='red', alpha=0.3)

    # Plot threshold
    if plot_threshold:
      plt.axhline(y=threshold, color='red', linestyle='--', label="Threshold")

    # Add labels, legend, and title
    plt.yscale('log')
    plt.xlabel('Time Index')
    plt.ylabel('Reconstruction Error')
    plt.legend()
    plt.title(f"Reconstruction Error Over Time{sufix}")
    plt.show()

# Grid Search Parameters - Benign Training

In [18]:
from sklearn.model_selection import ParameterGrid
from torch.optim import Adam
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## TransformerAutoencoder

In [ ]:
best_loss_params = {}
best_loss = {}
best_error_params = {}
best_error = {}
for window_size in window_sizes:
    best_loss_params[window_size] = None
    best_loss[window_size] = float('inf')
    best_error_params[window_size] = None
    best_error[window_size] = float('inf')

param_grid = {
    'window_size': window_sizes,         # Window sizes to test
    'embed_dim': [32, 64, 128, 256],     # Embedding dimension
    'num_heads': [2, 4, 8, 16],          # Number of heads
    'ff_dim': [128, 256, 512],           # Feedforward dimension
    'num_layers': [1, 2, 4, 6],          # Number of GRU layers
    'learning_rate': [1e-3, 1e-4, 1e-5], # Learning rates
    'epochs': [30],                         # Number of epochs (fixed for quick testing)
}

print(f"#"*(len('TransformerAutoencoder')+16))
print(f"###     TransformerAutoencoder     ###")
print(f"#"*(len('TransformerAutoencoder')+16))

for params in ParameterGrid(param_grid):     
    print(f"\tTesting parameters: {params}")
        
    # Initialize the model with current parameters
    model = TransformerAutoencoder(
        input_dim=input_dim,
        embed_dim=params['embed_dim'],
        num_heads=params['num_heads'],
        ff_dim=params['ff_dim'],
        num_layers=params['num_layers']
    ).to(device)
    
    model, total_loss, avg_rec_error = train_and_evaluate(model=model, 
                                                          train_loader=train_benign_loader[window_sizes.index(params['window_size'])], 
                                                          test_loader=test_loader[window_sizes.index(params['window_size'])], 
                                                          optimizer=Adam(model.parameters(), lr=params['learning_rate']),
                                                          epochs=params['epochs'], 
                                                          anomaly_weight=None, 
                                                          device=device)
    print(f"\tTotal Loss: {total_loss}, Average Reconstruction Error: {avg_rec_error}")

    if total_loss < best_loss[params['window_size']]:
        best_loss[params['window_size']] = total_loss
        best_loss_params[params['window_size']] = params

    if avg_rec_error < best_error[params['window_size']]:
        best_error[params['window_size']] = avg_rec_error
        best_error_params[params['window_size']] = params

for window_size in window_sizes:
    print(f"For Window Size = {window_size}:")
    print(f"   TOTAL LOSS -- Best Parameters: {best_loss_params[window_size]}, Best Total Loss: {best_loss[window_size]}")
    print(f"   AVERAGE RECONSTRUCTION ERROR -- Best Parameters: {best_error_params[window_size]}, Best Reconstruction Error: {best_error[window_size]}")

## ConvRecurrentAutoencoder

In [ ]:
best_loss_params = {}
best_loss = {}
best_error_params = {}
best_error = {}
for window_size in window_sizes:
    best_loss_params[window_size] = None
    best_loss[window_size] = float('inf')
    best_error_params[window_size] = None
    best_error[window_size] = float('inf')

param_grid = {
    'window_size': window_sizes,         # Window sizes to test
    'hidden_dim': [32, 64, 128, 256],    # Number of hidden dimensions
    'kernel_size': [3, 5, 7, 9],         # Kernel sizes for convolution
    'num_layers': [1, 2, 3, 6],          # Number of GRU layers
    'learning_rate': [1e-3, 5e-4, 1e-4], # Learning rates
    'epochs': [30],                         # Number of epochs (fixed for quick testing)
}

print(f"#"*(len('ConvRecurrentAutoencoder')+16))
print(f"###     ConvRecurrentAutoencoder     ###")
print(f"#"*(len('ConvRecurrentAutoencoder')+16))

for params in ParameterGrid(param_grid):    
    print(f"\tTesting parameters: {params}")
        
    # Initialize the model with current parameters
    model = ConvRecurrentAutoencoder(
           input_dim=input_dim,
           hidden_dim=params['hidden_dim'],
           kernel_size=params['kernel_size'],
           num_layers=params['num_layers']
    ).to(device)

    model, total_loss, avg_rec_error = train_and_evaluate(model=model, 
                                                          train_loader=train_benign_loader[window_sizes.index(params['window_size'])], 
                                                          test_loader=test_loader[window_sizes.index(params['window_size'])], 
                                                          optimizer=Adam(model.parameters(), lr=params['learning_rate']),
                                                          epochs=params['epochs'], 
                                                          anomaly_weight=None, 
                                                          device=device)
    print(f"\tTotal Loss: {total_loss}, Average Reconstruction Error: {avg_rec_error}")

    if total_loss < best_loss[params['window_size']]:
        best_loss[params['window_size']] = total_loss
        best_loss_params[params['window_size']] = params

    if avg_rec_error < best_error[params['window_size']]:
        best_error[params['window_size']] = avg_rec_error
        best_error_params[params['window_size']] = params

for window_size in window_sizes:
    print(f"For Window Size = {window_size}:")
    print(f"   TOTAL LOSS -- Best Parameters: {best_loss_params[window_size]}, Best Total Loss: {best_loss[window_size]}")
    print(f"   AVERAGE RECONSTRUCTION ERROR -- Best Parameters: {best_error_params[window_size]}, Best Reconstruction Error: {best_error[window_size]}")

## FullyConnectedAutoencoder

In [ ]:
best_loss_params = {}
best_loss = {}
best_error_params = {}
best_error = {}
for window_size in window_sizes:
    best_loss_params[window_size] = None
    best_loss[window_size] = float('inf')
    best_error_params[window_size] = None
    best_error[window_size] = float('inf')

param_grid = {
    'window_size': window_sizes,         # Window sizes to test
    'hidden_dim': [32, 64, 128, 256],    # Number of hidden dimensions
    'latent_dim': [16, 22, 32, 64],      # Latent dimension size
    'num_layers': [1, 2, 3, 4, 6],       # Number of GRU layers
    'learning_rate': [1e-3, 5e-4, 1e-4], # Learning rates
    'epochs': [30],                         # Number of epochs (fixed for quick testing)
}

print(f"#"*(len('FullyConnectedAutoencoder')+16))
print(f"###     FullyConnectedAutoencoder     ###")
print(f"#"*(len('FullyConnectedAutoencoder')+16))

for params in ParameterGrid(param_grid):
    print(f"\tTesting parameters: {params}")
        
    # Initialize the model with current parameters
    model = FullyConnectedAutoencoder(
        input_dim=input_dim,
        hidden_dim=params['hidden_dim'],
        latent_dim=params['latent_dim'],
        num_layers=params['num_layers']
    ).to(device)

    model, total_loss, avg_rec_error = train_and_evaluate(model=model, 
                                                          train_loader=train_benign_loader[window_sizes.index(params['window_size'])], 
                                                          test_loader=test_loader[window_sizes.index(params['window_size'])], 
                                                          optimizer=Adam(model.parameters(), lr=params['learning_rate']),
                                                          epochs=params['epochs'], 
                                                          anomaly_weight=None, 
                                                          device=device)
    print(f"\tTotal Loss: {total_loss}, Average Reconstruction Error: {avg_rec_error}")

    if total_loss < best_loss[params['window_size']]:
        best_loss[params['window_size']] = total_loss
        best_loss_params[params['window_size']] = params

    if avg_rec_error < best_error[params['window_size']]:
        best_error[params['window_size']] = avg_rec_error
        best_error_params[params['window_size']] = params

for window_size in window_sizes:
    print(f"For Window Size = {window_size}:")
    print(f"   TOTAL LOSS -- Best Parameters: {best_loss_params[window_size]}, Best Total Loss: {best_loss[window_size]}")
    print(f"   AVERAGE RECONSTRUCTION ERROR -- Best Parameters: {best_error_params[window_size]}, Best Reconstruction Error: {best_error[window_size]}")


# Evaluate Benign Models

In [20]:
import pandas as pd

anomaly_weight = None

## Train Models

In [21]:
import torch

models_path = 'models/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Best parameters for benign models
### NOTE: fill out for all window sizes based on the results of the grid search
best_benign_params = {
    1 : { 
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight}
    },
    2 : { 
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight}
    },
    8 : {
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight}
    },
    16 : { 
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight}
    },
    32 : { 
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': anomaly_weight}
    }
}

best_benign_models = {}
benign_rec_errs = {}
benign_smooth_rec_errs = {}
for window_size in window_sizes:
    best_benign_models[window_size], benign_rec_errs[window_size], benign_smooth_rec_errs[window_size] = train_all(models_path=models_path, 
                                                                                                                   models_type="benign", 
                                                                                                                   best_params=best_benign_params[window_size], 
                                                                                                                   anomaly_weight = {
                                                                                                                       'TransformerAutoencoder': best_benign_params[window_size]['TransformerAutoencoder']['anomaly_weight'],
                                                                                                                       'ConvRecurrentAutoencoder': best_benign_params[window_size]['ConvRecurrentAutoencoder']['anomaly_weight'],
                                                                                                                       'FullyConnectedAutoencoder': best_benign_params[window_size]['FullyConnectedAutoencoder']['anomaly_weight']
                                                                                                                       },
                                                                                                                   train_loader=train_benign_loader[window_sizes.index(window_size)], 
                                                                                                                   test_loader=test_loader[window_sizes.index(window_size)], 
                                                                                                                   window_size=window_size, 
                                                                                                                   input_dim=input_dim, 
                                                                                                                   device=device)



## Evaluate Models

In [ ]:

for window_size in window_sizes:
    for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
        model = best_benign_models[window_size][ae_name]
        rec_err = benign_rec_errs[window_size][ae_name]
        smooth_rec_errs = benign_smooth_rec_errs[window_size][ae_name]
        
        # Load total_loss
        errors = []
        colors = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "olive", "cyan", "magenta", "navy", "teal", "maroon", "gold"]
        for idx, filter_size in enumerate(filter_sizes):
            errors.append((smooth_rec_errs[filter_size], f"fs_{filter_size}", colors[idx]))
        
        plot_reconstruction_errors(attack_indices=list(range(len(test_attacks_indices))),
                                   attack_index_intervals=test_attacks_index_intervals,
                                   window_size = window_size,
                                   reconstruction_errors = errors,
                                   sufix = f" - {ae_name}, window size={window_size}")

In [ ]:
# Calculate model performance
model_performance_benign = {}
detected_attacks_benign = {}
precision = {}
recall = {}

for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
  print(f"#####   Calculate model performance for {ae_name}   #####")

  model_performance_benign[ae_name] = []
  detected_attacks_benign[ae_name] = []
  precision[ae_name] = []
  recall[ae_name] = []

  # Calculate model performance for all window sizes for all filter sizes
  for window_size in window_sizes:
    # Adjust indices to account for the window size - First-Point Strategy
    adjusted_indices = test_attacks_indices[:-window_size] 
    for filter_size in filter_sizes:
      model_performance_tmp, detected_attacks_tmp, precision_tmp, recall_tmp = calculate_model_performance(test_ground_truth_labels=adjusted_indices, 
                                                                                                           reconstruction_errors=benign_smooth_rec_errs[window_size][ae_name][filter_size], 
                                                                                                           method_name=f'{ae_name}_ws{window_size}_fs{filter_size}')
      model_performance_benign[ae_name].append(model_performance_tmp)
      detected_attacks_benign[ae_name].append(detected_attacks_tmp)
      precision[ae_name].append(precision_tmp)
      recall[ae_name].append(recall_tmp)


In [ ]:
pd.concat([
    pd.concat(model_performance_benign['TransformerAutoencoder']), 
    pd.concat(model_performance_benign['ConvRecurrentAutoencoder']),
    pd.concat(model_performance_benign['FullyConnectedAutoencoder'])
])

In [ ]:
detected_segments_for_ae_and_ws_fs = {}
real_segments_for_ae_and_ws_fs = {}
removed_real_segments_for_ae_and_ws_fs = {}

for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:

  detected_segments_for_ae_and_ws_fs[ae_name] = {}
  real_segments_for_ae_and_ws_fs[ae_name] = {}
  removed_real_segments_for_ae_and_ws_fs[ae_name] = {}
  
  print("#"*len(f'#####   {ae_name}   #####'))
  print(f"#####   {ae_name}   #####")
  print("#"*len(f'#####   {ae_name}   #####'))
  for idx1, window_size in enumerate(window_sizes):

    detected_segments_for_ae_and_ws_fs[ae_name][window_size] = {}
    real_segments_for_ae_and_ws_fs[ae_name][window_size] = {}
    removed_real_segments_for_ae_and_ws_fs[ae_name][window_size] = {}

    print(f"#####")
    print(f"##### window_size: {window_size} #####")
    print(f"#####")
    for idx2, filter_size in enumerate(filter_sizes):
      print(f"###")
      print(f"### filter_size: {filter_size} ###")
      print(f"###")

      detected_segments = find_segments(detected_attacks_benign[ae_name][len(filter_sizes)*idx1+idx2])
      real_segments = find_segments(test_attacks_indices[:-window_size])
      removed_real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size] = 0
      print(f"Detected segments: {detected_segments}")
      print(f"Real segments: {real_segments}")

      i = 0
      j = 0
      detected_segments_tmp = []
      real_segments_tmp = []
      while i<len(detected_segments) and j<len(real_segments):
          if detected_segments[i][1] <= real_segments[j][0]:
            i+=1
          elif (detected_segments[i][0] < real_segments[j][1]) and (real_segments[j][0] < detected_segments[i][1]):
            detected_segments_tmp.append(detected_segments[i])
            real_segments_tmp.append(real_segments[j])
            i+=1
            j+=1
            while i<len(detected_segments) and j<len(real_segments) and (detected_segments[i][1] < real_segments[j][0]):
              i+=1
            while i<len(detected_segments) and j<len(real_segments) and (real_segments[j][1] < detected_segments[i][0]):
              removed_real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]+=1
              j+=1
          elif real_segments[j][1] <= detected_segments[i][0] :
            removed_real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]+=1
            j+=1

      detected_attacks_tmp  = np.zeros(detected_attacks_benign[ae_name][len(filter_sizes)*idx1+idx2].shape[0])
      for attack in detected_segments_tmp:
          detected_attacks_tmp[attack[0]:attack[1]+1] = 1
      real_attacks_tmp  = np.zeros(test_attacks_indices[:-window_size].shape[0])
      for attack in real_segments_tmp:
          real_attacks_tmp[attack[0]:attack[1]+1] = 1

      detected_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size] = find_segments(detected_attacks_tmp)
      real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size] = find_segments(real_attacks_tmp)
      print(f"Detected segments to compare: {detected_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]}")
      print(f"Real segments to compare: {real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]}")
      print(f"Removed real segments: {removed_real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]}")


In [ ]:
for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
    
    for idx1, window_size in enumerate(window_sizes):
        for idx2, filter_size in enumerate(filter_sizes):
        
            detected_attacks_len = len(find_segments(detected_attacks_benign[ae_name][len(filter_sizes)*idx1+idx2]))
            detected_segments = detected_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]
            method = f'{ae_name}_ws{window_size}_fs{filter_size}'
            results = model_performance_benign[ae_name][len(filter_sizes)*idx1+idx2]

            print("#"*len(f'#####   {method}   #####'))
            print(f"#####   {method}   #####")
            print("#"*len(f'#####   {method}   #####'))
            
            real_segments = real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]
            assert len(detected_segments) == len(real_segments), f"Segments lists must have the same length [{len(detected_segments)}<>{len(real_segments)}]."

            results['num_of_detected'] = detected_attacks_len

            ttd = 0
            for detected_segment, real_segment in zip(detected_segments, real_segments):
                ttd = ttd + max((detected_segment[0] - real_segment[0]), 0)/real_segment[2]
            ttd = ttd + removed_real_segments_for_ae_and_ws_fs[ae_name][window_size][filter_size]
            Sttd = 1-ttd/len(find_segments(test_attacks_indices[:-window_size]))

            gama = 0.5
            Sclf = results['Sclf'].values[0]
            S = gama * Sttd + (1-gama) * Sclf

            print(f"Sttd = {Sttd}, S = {S}")
            results['Sttd'] = Sttd
            results['S'] = S


## Summary

In [ ]:
pd.concat([
    pd.concat(model_performance_benign['TransformerAutoencoder']), 
    pd.concat(model_performance_benign['ConvRecurrentAutoencoder']),
    pd.concat(model_performance_benign['FullyConnectedAutoencoder'])
])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# visualize
f, ax = plt.subplots(1,figsize=(7,7))

lw = 3
font_size = 15

ls = 	'-'

for ae_name, ae_label, color in [('TransformerAutoencoder', "TransAE", "blue"), 
                                 ('ConvRecurrentAutoencoder', "ConvRecAE", "red"), 
                                 ('FullyConnectedAutoencoder', "FCAE", "green")]:
    ax.plot(model_performance_benign[ae_name][0]["fpr"].iloc[0], 
            model_performance_benign[ae_name][0]["tpr"].iloc[0], 
            linewidth=lw, linestyle= ls, label='{}'.format(f"{ae_label}, ws=1"), color=color, alpha=0.2)
    ax.plot(model_performance_benign[ae_name][1]["fpr"].iloc[0], 
            model_performance_benign[ae_name][1]["tpr"].iloc[0], 
            linewidth=lw, linestyle= ls, label='{}'.format(f"{ae_label}, ws=2"), color=color, alpha=0.4)
    ax.plot(model_performance_benign[ae_name][2]["fpr"].iloc[0], 
            model_performance_benign[ae_name][2]["tpr"].iloc[0], 
            linewidth=lw, linestyle= ls, label='{}'.format(f"{ae_label}, ws=8"), color=color, alpha=0.6)
    ax.plot(model_performance_benign[ae_name][3]["fpr"].iloc[0], 
            model_performance_benign[ae_name][3]["tpr"].iloc[0], 
            linewidth=lw, linestyle= ls, label='{}'.format(f"{ae_label}, ws=16"), color=color, alpha=0.8)
    ax.plot(model_performance_benign[ae_name][4]["fpr"].iloc[0], 
            model_performance_benign[ae_name][4]["tpr"].iloc[0], 
            linewidth=lw, linestyle= ls, label='{}'.format(f"{ae_label}, ws=32"), color=color, alpha=1)

# axes label size
ax.set_xlabel("False Positive Rate", fontsize=font_size)
ax.set_ylabel("True Positive Rate", fontsize=font_size)

# Tick label size
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)

# set y lim
ax.set_ylim(0.495, 1.005)

ax.legend(fontsize=font_size)
# Legend 1: Based on color (AE)
legend1 = ax.legend(
    handles=[
        Line2D([0], [0], color='blue', lw=lw, label=f'Trans'),
        Line2D([0], [0], color='red', lw=lw, label=f'ConvRec'),
        Line2D([0], [0], color='green', lw=lw, label=fr'FC'),
    ],
    loc='center right',
    title='Autoencoders',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend1.get_title():  # Check if there is a title
    legend1.get_title().set_fontsize(font_size) # Use the font_size variable
ax.add_artist(legend1)  # Add the first legend manually

# Legend 2: Based on color intensity (ws)
intensities = [0.2, 0.4, 0.6, 0.8, 1.0]
legend2 = ax.legend(
    handles=[
        Patch(facecolor='gray', alpha=alpha, label=f'{ws}')
        for ws, alpha in [(1, 0.2),(2, 0.4),(8, 0.6),(16, 0.8),(32, 1.0)]
    ],
    loc='lower right',
    title='window size',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend2.get_title():  # Check if there is a title
    legend2.get_title().set_fontsize(font_size) # Use the font_size variable


ax.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# visualize
f, ax = plt.subplots(1,figsize=(7,7))

lw = 3
font_size = 15


for ae_name, ae_label, color in [('TransformerAutoencoder', "TransAE", "blue"), 
                                 ('ConvRecurrentAutoencoder', "ConvRecAE", "red"), 
                                 ('FullyConnectedAutoencoder', "FCAE", "green")]:
    ax.plot(model_performance_benign[ae_name][15*2+12]["fpr"].iloc[0], 
            model_performance_benign[ae_name][15*2+12]["tpr"].iloc[0], 
            linewidth=lw, linestyle= None, label='{}'.format(f"{ae_label}, fs=25"), color=color, alpha=1)
    ax.plot(model_performance_benign[ae_name][15*2+11]["fpr"].iloc[0], 
            model_performance_benign[ae_name][15*2+11]["tpr"].iloc[0], 
            linewidth=lw, linestyle= None, label='{}'.format(f"{ae_label}, fs=23"), color=color, alpha=0.8)
    ax.plot(model_performance_benign[ae_name][15*2+10]["fpr"].iloc[0], 
            model_performance_benign[ae_name][15*2+10]["tpr"].iloc[0], 
            linewidth=lw, linestyle= None, label='{}'.format(f"{ae_label}, fs=21"), color=color, alpha=0.6)
    ax.plot(model_performance_benign[ae_name][15*2+9]["fpr"].iloc[0], 
            model_performance_benign[ae_name][15*2+9]["tpr"].iloc[0], 
            linewidth=lw, linestyle= None, label='{}'.format(f"{ae_label}, fs=19"), color=color, alpha=0.3)
    ax.plot(model_performance_benign[ae_name][15*2+8]["fpr"].iloc[0], 
            model_performance_benign[ae_name][15*2+8]["tpr"].iloc[0], 
            linewidth=lw, linestyle= None, label='{}'.format(f"{ae_label}, fs=17"), color=color, alpha=0.1)

# axes label size
ax.set_xlabel("False Positive Rate", fontsize=font_size)
ax.set_ylabel("True Positive Rate", fontsize=font_size)

# Tick label size
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)

# set y lim
ax.set_ylim(0.955, 1.005)

ax.legend(fontsize=font_size)
# Legend 1: Based on color (AE)
legend1 = ax.legend(
    handles=[
        Line2D([0], [0], color='blue', lw=lw, label=f'Trans'),
        Line2D([0], [0], color='red', lw=lw, label=f'ConvRec'),
        Line2D([0], [0], color='green', lw=lw, label=fr'FC'),
    ],
    loc='center right',
    title='Autoencoders',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend1.get_title():  # Check if there is a title
    legend1.get_title().set_fontsize(font_size) # Use the font_size variable
ax.add_artist(legend1)  # Add the first legend manually

# Legend 2: Based on color intensity (fs)
legend2 = ax.legend(
    handles=[
        Patch(facecolor='gray', alpha=alpha, label=f'{fs}')
        for fs, alpha in [(17, 0.1),(19, 0.3),(21, 0.6),(23, 0.8),(25, 1.0)]
    ],
    loc='lower right',
    title='filter size',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend2.get_title():  # Check if there is a title
    legend2.get_title().set_fontsize(font_size) # Use the font_size variable
ax.add_artist(legend2)  # Add the first legend manually

legend3 = ax.legend(
    handles=[],
    loc='upper left',
    title='ws=8',
)
# Set the font size of the legend title after creation
if legend3.get_title():  # Check if there is a title
    legend3.get_title().set_fontsize(font_size) # Use the font_size variable


ax.grid(True)
plt.show()

# Evaluate Attackaware Models

In [60]:
import pandas as pd

anomaly_weights = [-0.1,-0.2,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1]
window_size = 8 # We fix window size to 8 for attackaware models
filter_size = 21 # We fix filter size to 21 for attackaware models

## Train Models

In [61]:
import torch

models_path = 'models/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Best parameters for attackaware models
best_attackaware_params = {
    ### NOTE: fill out for window size = 8 based on the results of the grid search
    8 : { 
        # 'TransformerAutoencoder': {'embed_dim': ..., 'epochs': ..., 'ff_dim': ..., 'learning_rate': ..., 'num_heads': ..., 'num_layers': ..., 'anomaly_weight': ...},
        # 'ConvRecurrentAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'kernel_size': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': ...},
        # 'FullyConnectedAutoencoder': {'epochs': ..., 'hidden_dim': ..., 'latent_dim': ..., 'learning_rate': ..., 'num_layers': ..., 'anomaly_weight': ...}
    },
}

best_attackaware_models = {}
attackaware_rec_errs = {}
attackaware_smooth_rec_errs = {}

for anomaly_weight in anomaly_weights:
    best_attackaware_models[anomaly_weight], attackaware_rec_errs[anomaly_weight], attackaware_smooth_rec_errs[anomaly_weight] = train_all(models_path=models_path, 
                                                                                                                                           models_type="attackaware", 
                                                                                                                                           best_params=best_attackaware_params[window_size], 
                                                                                                                                           anomaly_weight = {
                                                                                                                                               'TransformerAutoencoder': anomaly_weight,
                                                                                                                                               'ConvRecurrentAutoencoder': anomaly_weight,
                                                                                                                                               'FullyConnectedAutoencoder': anomaly_weight
                                                                                                                                               },
                                                                                                                                           train_loader=train_attack_loader[window_sizes.index(window_size)],
                                                                                                                                           test_loader=test_loader[window_sizes.index(window_size)], 
                                                                                                                                           window_size=window_size, 
                                                                                                                                           input_dim=input_dim, 
                                                                                                                                           device=device)



## Evaluate Models

In [ ]:

for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:

    # Load total_loss
    errors = []
    colors = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "olive", "cyan", "magenta", "navy", "teal", "maroon", "gold"]

    for idx, anomaly_weight in enumerate(anomaly_weights):
        model = best_attackaware_models[anomaly_weight][ae_name]
        rec_err = attackaware_rec_errs[anomaly_weight][ae_name]
        smooth_rec_errs = attackaware_smooth_rec_errs[anomaly_weight][ae_name]
        
        errors.append((smooth_rec_errs[filter_size], f"aw_{anomaly_weight}", colors[idx]))
        
    plot_reconstruction_errors(attack_indices=list(range(len(test_attacks_indices))),
                               attack_index_intervals=test_attacks_index_intervals,
                               window_size = window_size,
                               reconstruction_errors = errors,
                               sufix = f" - {ae_name}, window size={window_size}, filter size={filter_size}")


In [ ]:
# Calculate model performance
model_performance_attackaware = {}
detected_attacks_attackaware = {}
precision = {}
recall = {}

for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
  print(f"#####   Calculate model performance for {ae_name}   #####")

  model_performance_attackaware[ae_name] = []
  detected_attacks_attackaware[ae_name] = []
  precision[ae_name] = []
  recall[ae_name] = []

  adjusted_indices = test_attacks_indices[:-window_size] 

  # Calculate model performance for all anomaly weights for the fixed window size and filter size
  for idx, anomaly_weight in enumerate(anomaly_weights):
      
      model_performance_tmp, detected_attacks_tmp, precision_tmp, recall_tmp = calculate_model_performance(test_ground_truth_labels=adjusted_indices, 
                                                                                                           reconstruction_errors=attackaware_smooth_rec_errs[anomaly_weight][ae_name][filter_size], 
                                                                                                           method_name=f'{ae_name}_ws{window_size}_fs{filter_size}_aw{anomaly_weight}')
      model_performance_attackaware[ae_name].append(model_performance_tmp)
      detected_attacks_attackaware[ae_name].append(detected_attacks_tmp)
      precision[ae_name].append(precision_tmp)
      recall[ae_name].append(recall_tmp)


In [ ]:
pd.concat([
    pd.concat(model_performance_attackaware['TransformerAutoencoder']), 
    pd.concat(model_performance_attackaware['ConvRecurrentAutoencoder']),
    pd.concat(model_performance_attackaware['FullyConnectedAutoencoder'])
])

In [ ]:
detected_segments_for_ae_and_ws_fs = {}
real_segments_for_ae_and_ws_fs = {}
removed_real_segments_for_ae_and_ws_fs = {}

for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:

  detected_segments_for_ae_and_ws_fs[ae_name] = {}
  real_segments_for_ae_and_ws_fs[ae_name] = {}
  removed_real_segments_for_ae_and_ws_fs[ae_name] = {}
  
  print("#"*len(f'#####   {ae_name}   #####'))
  print(f"#####   {ae_name}   #####")
  print("#"*len(f'#####   {ae_name}   #####'))
  for idx, anomaly_weight in enumerate(anomaly_weights):
      print("#"*len(f'###   {anomaly_weight}   ###'))
      print(f"###   {anomaly_weight}   ###")
      print("#"*len(f'###   {anomaly_weight}   ###'))
   
      detected_segments = find_segments(detected_attacks_attackaware[ae_name][idx])
      real_segments = find_segments(test_attacks_indices[:-window_size])
      removed_real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight] = 0
      print(f"Detected segments: {detected_segments}")
      print(f"Real segments: {real_segments}")

      i = 0
      j = 0
      detected_segments_tmp = []
      real_segments_tmp = []
      while i<len(detected_segments) and j<len(real_segments):
          if detected_segments[i][1] <= real_segments[j][0]:
            i+=1
          elif (detected_segments[i][0] < real_segments[j][1]) and (real_segments[j][0] < detected_segments[i][1]):
            detected_segments_tmp.append(detected_segments[i])
            real_segments_tmp.append(real_segments[j])
            i+=1
            j+=1
            while i<len(detected_segments) and j<len(real_segments) and (detected_segments[i][1] < real_segments[j][0]):
              i+=1
            while i<len(detected_segments) and j<len(real_segments) and (real_segments[j][1] < detected_segments[i][0]):
              removed_real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]+=1
              j+=1
          elif real_segments[j][1] <= detected_segments[i][0]:
            removed_real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]+=1
            j+=1
      detected_attacks_tmp  = np.zeros(detected_attacks_attackaware[ae_name][idx].shape[0])
      for attack in detected_segments_tmp:
          detected_attacks_tmp[attack[0]:attack[1]+1] = 1
      real_attacks_tmp  = np.zeros(test_attacks_indices[:-window_size].shape[0])
      for attack in real_segments_tmp:
          real_attacks_tmp[attack[0]:attack[1]+1] = 1

      detected_segments_for_ae_and_ws_fs[ae_name][anomaly_weight] = find_segments(detected_attacks_tmp)
      real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight] = find_segments(real_attacks_tmp)
      print(f"Detected segments to compare: {detected_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]}")
      print(f"Real segments to compare: {real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]}")
      print(f"Removed real segments: {removed_real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]}")

In [ ]:
for ae_name in ['TransformerAutoencoder', 'ConvRecurrentAutoencoder', 'FullyConnectedAutoencoder']:
    
    for idx, anomaly_weight in enumerate(anomaly_weights):
        
        detected_attacks_len = len(find_segments(detected_attacks_attackaware[ae_name][idx]))
        detected_segments = detected_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]
        method = f'{ae_name}_ws{window_size}_fs{filter_size}'
        results = model_performance_attackaware[ae_name][idx]

        print("#"*len(f'#####   {method}   #####'))
        print(f"#####   {method}   #####")
        print("#"*len(f'#####   {method}   #####'))
            
        real_segments = real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]
        assert len(detected_segments) == len(real_segments), f"Segments lists must have the same length [{len(detected_segments)}<>{len(real_segments)}]."

        results['num_of_detected'] = detected_attacks_len

        ttd = 0
        for detected_segment, real_segment in zip(detected_segments, real_segments):
            ttd = ttd + max((detected_segment[0] - real_segment[0]), 0)/real_segment[2]
        ttd = ttd + removed_real_segments_for_ae_and_ws_fs[ae_name][anomaly_weight]
        Sttd = 1-ttd/len(find_segments(test_attacks_indices[:-window_size]))

        gama = 0.5
        Sclf = results['Sclf'].values[0]
        S = gama * Sttd + (1-gama) * Sclf

        print(f"Sttd = {Sttd}, S = {S}")
        results['Sttd'] = Sttd
        results['S'] = S


## Summary

In [ ]:
pd.concat([
    pd.concat(model_performance_attackaware['TransformerAutoencoder']), 
    pd.concat(model_performance_attackaware['ConvRecurrentAutoencoder']),
    pd.concat(model_performance_attackaware['FullyConnectedAutoencoder'])
])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import matplotlib.pyplot as plt

# visualize
f, ax = plt.subplots(1,figsize=(7,7))

lw = 3
font_size = 15

ax.plot(model_performance_benign['TransformerAutoencoder'][15*2+12]["fpr"].iloc[0], 
        model_performance_benign['TransformerAutoencoder'][15*2+12]["tpr"].iloc[0], 
        linestyle='--', linewidth=lw, label='{}'.format("Benign TransAE, fs=21"), color="blue")
ax.plot(model_performance_benign['ConvRecurrentAutoencoder'][15*2+12]["fpr"].iloc[0], 
        model_performance_benign['ConvRecurrentAutoencoder'][15*2+12]["tpr"].iloc[0], 
        linestyle='--', linewidth=lw, label='{}'.format("Benign ConvRecAE, fs=21"), color="red")
ax.plot(model_performance_benign['FullyConnectedAutoencoder'][15*2+12]["fpr"].iloc[0], 
        model_performance_benign['FullyConnectedAutoencoder'][15*2+12]["tpr"].iloc[0], 
        linestyle='--', linewidth=lw, label='{}'.format("Benign FCAE, fs=21"), color="green")
ax.plot(model_performance_attackaware['FullyConnectedAutoencoder'][4]["fpr"].iloc[0], 
        model_performance_attackaware['FullyConnectedAutoencoder'][4]["tpr"].iloc[0], 
        linestyle = None, linewidth=lw, label='{}'.format("Attack aware TransAE, fs=21"), color="blue")
ax.plot(model_performance_attackaware['FullyConnectedAutoencoder'][4]["fpr"].iloc[0], 
        model_performance_attackaware['FullyConnectedAutoencoder'][4]["tpr"].iloc[0], 
        linestyle = None, linewidth=lw, label='{}'.format("Attack aware ConvRecAE, fs=21"), color="red")
ax.plot(model_performance_attackaware['FullyConnectedAutoencoder'][4]["fpr"].iloc[0], 
        model_performance_attackaware['FullyConnectedAutoencoder'][4]["tpr"].iloc[0], 
        linestyle = None, linewidth=lw, label='{}'.format("Attack aware FCAE, fs=21"), color="green")

# axes label size
ax.set_xlabel("False Positive Rate", fontsize=font_size)
ax.set_ylabel("True Positive Rate", fontsize=font_size)

# Tick label size
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)

# set y lim
ax.set_ylim(0.979, 1.001)

# Legend 1: Based on color (Category)
legend1 = ax.legend(
    handles=[
        Line2D([0], [0], color='blue', lw=lw, label=f'TransAE'),
        Line2D([0], [0], color='red', lw=lw, label=f'ConvRecAE'),
        Line2D([0], [0], color='green', lw=lw, label=fr'FCAE'),
    ],
    loc='lower right',
    title='Attacked data',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend1.get_title():  # Check if there is a title
    legend1.get_title().set_fontsize(font_size) # Use the font_size variable
ax.add_artist(legend1)  # Add the first legend manually

# Legend 2: Based on linestyle (Method)
legend2 = ax.legend(
    handles=[
        Line2D([0], [0], color='blue', linestyle='--', lw=lw, label=f'TransAE'),
        Line2D([0], [0], color='red', linestyle='--', lw=lw, label=f'ConvRecAE'),
        Line2D([0], [0], color='green', linestyle='--', lw=lw, label=f'FCAE'),
    ],
    loc='center right',
    title='Benign data',
    fontsize=font_size
)
# Set the font size of the legend title after creation
if legend2.get_title():  # Check if there is a title
    legend2.get_title().set_fontsize(font_size) # Use the font_size variable
ax.add_artist(legend2)  # Add the first legend manually


legend3 = ax.legend(
    handles=[],
    loc='upper right',
    title='ws=8, fs=21',
)
# Set the font size of the legend title after creation
if legend3.get_title():  # Check if there is a title
    legend3.get_title().set_fontsize(font_size) # Use the font_size variable

ax.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.lines import Line2D

len_test_datetime = 2089
start_date = '2017-04-01 00:00:00'
hourly_datetime_list = pd.date_range(start=start_date, periods=len_test_datetime, freq='h')[:-window_size]

# load total_loss
errors = [
      (benign_smooth_rec_errs[window_size]['TransformerAutoencoder'][filter_size], f"TransAE - ws=8, fs=21, benign data", "blue"),
      (benign_smooth_rec_errs[window_size]['ConvRecurrentAutoencoder'][filter_size], f"ConvRecAE - ws=8, fs=21, benign data", "red"),
      (benign_smooth_rec_errs[window_size]['FullyConnectedAutoencoder'][filter_size], f"FCAE - ws=8, fs=21, benign data", "green"),
      (attackaware_smooth_rec_errs[-0.2]['TransformerAutoencoder'][filter_size], f"TransAE - ws=8, fs=21, aw=-0.2, attacked data", "blue"),
      (attackaware_smooth_rec_errs[-0.2]['ConvRecurrentAutoencoder'][filter_size], f"ConvRecAE - ws=8, fs=21, aw=-0.2, attacked data", "red"),
      (attackaware_smooth_rec_errs[-0.2]['FullyConnectedAutoencoder'][filter_size], f"FCAE - ws=8, fs=21, aw=-0.2, attacked data", "green"),
]


# Plotting both reconstruction errors on a single plot
plt.figure(figsize=(24, 4))

lw = 3
font_size = 15

for (errors, label, color) in errors:
    if 'benign' in label:
        plt.plot(hourly_datetime_list, errors, linewidth=lw, label=label, color=color, linestyle='--')
    else:
        plt.plot(hourly_datetime_list, errors, linewidth=lw, label=label, color=color, linestyle='-')

# Shading attack intervals
for interval in test_attacks_index_intervals:
    start_idx = interval["start"]
    end_idx = interval["end"]

    # Adjust the indices to account for the window size
    if start_idx >= window_size and end_idx >= window_size:
        adjusted_start_idx = start_idx - window_size
        adjusted_end_idx = end_idx - window_size
        plt.axvspan(hourly_datetime_list[adjusted_start_idx], hourly_datetime_list[adjusted_end_idx], color='red', alpha=0.3)
    elif start_idx < window_size and end_idx >= window_size:
        adjusted_start_idx = 0
        adjusted_end_idx = end_idx - window_size
        plt.axvspan(hourly_datetime_list[adjusted_start_idx], hourly_datetime_list[adjusted_end_idx], color='red', alpha=0.3)


# Add labels, legend, and title
plt.yscale('log')
plt.xlabel("Date", fontsize=font_size)
plt.ylabel("Reconstruction Error", fontsize=font_size)

# Tick label size
plt.tick_params(axis='x', labelsize=font_size)
plt.tick_params(axis='y', labelsize=font_size)

 # First legend: Colors
legend1_handles = [
       Line2D([0], [0], color='blue', lw=lw, label='TransAE'),
       Line2D([0], [0], color='red', lw=lw, label='ConvRecAE'),
       Line2D([0], [0], color='green', lw=lw, label='FCAE'),
]
legend1 = plt.legend(handles=legend1_handles, loc='upper left', title='Attacked data', fontsize=font_size)
# Set the font size of the legend title after creation
if legend1.get_title():  # Check if there is a title
        legend1.get_title().set_fontsize(font_size) # Use the font_size variable
plt.gca().add_artist(legend1)  # Add manually to keep it

# Second legend: Styles
legend2_handles = [
       Line2D([0], [0], color='blue', linestyle='--', lw=lw, label='TransAE'),
       Line2D([0], [0], color='red', linestyle='--', lw=lw, label='ConvRecAE'),
       Line2D([0], [0], color='green', linestyle='--', lw=lw, label='FCAE'),
]
legend2 = plt.legend(handles=legend2_handles, loc='upper right', title='Benign data', fontsize=font_size)
# Set the font size of the legend title after creation
if legend2.get_title():  # Check if there is a title
        legend2.get_title().set_fontsize(font_size) # Use the font_size variable

plt.title(f"Reconstruction Error Over Time - test data with realistic attacks (window size={window_size}, filter size={filter_size}, anomaly weight = {0.2})", fontsize=font_size)
plt.show()


# Compare to SOA

In [ ]:
results = {
    "Housh_and_Ohar": {
        "num_of_detected": 7,
        "tp": 388,
        "fp": 5,
        "tn": 1677,
        "fn": 19,
        "tpr": 0.953,  #  TPR = TP/(TP+FN) => Recall
        "tnr": 0.997,  #  TNR = TN/(FP+TN)
        "S": 0.970,
        "Sttd": 0.965,
        "Sclf": 0.975,
    },
    "Abokifa_et_al": {
        "num_of_detected": 7,
        "tp": 375,
        "fp": 69,
        "tn": 1613,
        "fn": 32,
        "tpr": 0.921,
        "tnr": 0.959,
        "S": 0.949,
        "Sttd": 0.958,
        "Sclf": 0.940,
    },
    "Giacomoni_et_al": {
        "num_of_detected": 7,
        "tp": 341,
        "fp": 5,
        "tn": 1677,
        "fn": 66,
        "tpr": 0.838,
        "tnr": 0.997,
        "S": 0.927,
        "Sttd": 0.936,
        "Sclf": 0.917,
    },
    "Brentan_et_al": {
        "num_of_detected": 6,
        "tp": 362,
        "fp": 45,
        "tn": 1637,
        "fn": 45,
        "tpr": 0.889,
        "tnr": 0.973,
        "S": 0.894,
        "Sttd": 0.857,
        "Sclf": 0.931,
    },
    "Chandy_et_al": {
        "num_of_detected": 7,
        "tp": 349,
        "fp": 541,
        "tn": 1141,
        "fn": 58,
        "tpr": 0.857,
        "tnr": 0.678,
        "S": 0.802,
        "Sttd": 0.835,
        "Sclf": 0.768,
    },
    "Pasha_et_al": {
        "num_of_detected": 7,
        "tp": 134,
        "fp": 14,
        "tn": 1668,
        "fn": 273,
        "tpr": 0.329,
        "tnr": 0.992,
        "S": 0.773,
        "Sttd": 0.885,
        "Sclf": 0.660,
    },
    "Aghashahi_et_al": {
        "num_of_detected": 3,
        "tp": 161,
        "fp": 195,
        "tn": 1487,
        "fn": 246,
        "tpr": 0.396,
        "tnr": 0.884,
        "S": 0.534,
        "Sttd": 0.429,
        "Sclf": 0.640,
    },
    "Stojanovic_et_al": {
        "num_of_detected": 7,
        "tp": 385,
        "fp": 48,
        "tn": 1628,
        "fn": 22,
        "tpr": 0.9459,
        "tnr": 0.9717,
        "S": 0.9571,
        "Sttd": 0.9556,
        "Sclf": 0.9587,
    },
}

results_df = pd.DataFrame.from_dict(results, orient='index')
results_df['total_attacks'] = None
results_df['total_clean'] = None
results_df['missed_attacks'] = None
results_df['wrongly_detected'] = None

for index, row in results_df.iterrows():
  # Calculate the percentage of missed attack samples (False Negatives)
  total_attack_samples = row["tp"] + row["fn"]
  missed_attacks = row["fn"]
  missed_attack_percentage = (missed_attacks / total_attack_samples) * 100

  # Calculate the percentage of wrongly detected samples (False Positives)
  total_clean_samples = row["fp"] + row["tn"]
  wrongly_detected = row["fp"]
  wrongly_detected_percentage = (wrongly_detected / total_clean_samples) * 100

  results_df.loc[index, 'total_attacks'] = total_attack_samples
  results_df.loc[index, 'total_clean'] = total_clean_samples
  results_df.loc[index, 'missed_attacks'] = missed_attack_percentage
  results_df.loc[index, 'wrongly_detected'] = wrongly_detected_percentage

results_df
